# Tables Total Recall

The purpose of this notebook is to get to the point where we can extract 100% of part names from transistor hardware sheets.

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
load_pickle = True # This takes 12sec
save_pickle = True # Saved all the documents last time. Took 30min.

corpus_loaded = False
if load_pickle:
    try:
        import cPickle
        with open("data/hardware/hardware_corpus.pkl","r") as pkl:
            %time corpus = cPickle.load(pkl)
        corpus_loaded = True
        print "Corpus has been loaded."
    except:
        print "Corpus could not be loaded."
        print "Corpus will be parsed instead..."
if not corpus_loaded:
    from snorkel.parser import CorpusParser
    from snorkel.parser import HTMLParser
#     from snorkel.parser import SentenceParser
    from snorkel.parser import TableParser

    doc_parser = HTMLParser(path='data/hardware/hardware_html/')
#     context_parser = SentenceParser()
    context_parser = TableParser()

    cp = CorpusParser(doc_parser, context_parser)
    %time corpus = cp.parse_corpus(name='Hardware Corpus')
    print "Corpus has been parsed."
    
    if save_pickle:
        with open("data/hardware/hardware_corpus.pkl","w") as pkl:
            %time cPickle.dump(corpus, pkl)
            print "Corpus has been pickled."

CPU times: user 11.9 s, sys: 64 ms, total: 12 s
Wall time: 12 s
Corpus has been loaded.


In [25]:
from load_dictionaries import load_hardware_dictionary

gold_parts = load_hardware_dictionary()
print "Loaded %s gold part numbers." % len(gold_parts)

Loaded 179 gold part numbers.


In [26]:
from snorkel.candidates import Ngrams, EntityExtractor
from snorkel.matchers import RegexMatchEach, DictionaryMatch, RangeMatcher

ngrams = Ngrams(n_max=2)
part_matcher = DictionaryMatch(d=gold_parts, longest_match_only=False)
part_extractor = EntityExtractor(ngrams, part_matcher)

%time parts = part_extractor.extract(corpus.get_sentences(), name='all')
for p in parts[:5]: 
    print p
print "Extracted %s candidate part numbers." % len(parts)

CPU times: user 1.28 s, sys: 0 ns, total: 1.28 s
Wall time: 1.25 s
Span("BC546", context=None, chars=[41,45], words=[18,18])
Span("BC547", context=None, chars=[50,54], words=[22,22])
Span("BC548", context=None, chars=[59,63], words=[26,26])
Span("BC546B", context=None, chars=[24,29], words=[11,11])
Span("BC547B", context=None, chars=[32,37], words=[13,13])
Extracted 1111 candidate part numbers.


In [27]:
from fractions import Fraction
g = set(gold_parts)
x = set([p.get_span() for p in parts])
recall = Fraction(len(g.intersection(x)),len(g))
precision = Fraction(len(g.intersection(x)),len(x))
print "Recall: %0.3f (%s/%s)" % (float(recall), recall.numerator, recall.denominator)
print "Precision: %0.3f (%s/%s)" % (float(precision), precision.numerator, precision.denominator)

Recall: 0.799 (143/179)
Precision: 1.000 (1/1)


In [28]:
print x - g
for p in x:
    print p

set([])
BC817-16
2N4123
2N4124
BC818-16
BC857CW
BC817K-40W
BC859C
BC859B
BC817W
BC550C
BC817K-25
SMBT3904S
DTC124EE
BC846AW
BC847CW
BC846
BC847
BC850B
BC850C
BC846A
BC846B
BC858C-3L
BC857A-Z3E
BC238B
BC858B
BC858C
BC846W
BC858A
BC327-25
BC857B-3F
DTC123JE
BC846BW
BC807-40
BC847BW
BC858A-3J
2N3906
2N3904
2N3905
DTC123EE
BC182B
BC860BW
DTC114WKA
BC857BL3
BC327-16
BC856A-3A
DTC114EE
BC817-40
BC817-40W
DTC143ZE
BC817-25
BC337-40
BC859C-Z4C
BC859A-Z4A
BC860C-4GZ
BC857BW
DTC114YE
BC860CW
BC817K-40
BC860B
DTC124XE
BC807-16W
BC807-25
BC818
BC807-25W
BC817K-16W
BC858B-3K
BC817
BC859B-4B
BC849B
BC849C
DTC114TE
BC549C
BC549B
DTC143EE
BC818K-16W
MMBT3904
MMBT3906
BC860A-Z4E
BC549
BC548
BC807-16
DTC143TE
BC807W
BC547
BC546
BC846T
BC807
MMBT6427
BC327
BC550
BC548A
BC548B
BC548C
BC818K-40
BC857C-3G
BC817-16W
BC858CW
DTC114WUA
BC847B
BC847C
DTC144EE
BC847A
PZT3904
BC337-16
PZT3906
BC857C
BC857B
BC857A
BC337-25
BC817K-25W
BC182A
BC337
BC338
BC547A
BC547C
BC547B
BC239
BC856B-Z3B
BC237A
BC847AW
BC817-25W

Fraction(1, 1)

In [45]:
# ranges (e.g., BC546-BC548) or lists (e.g., BC546/547/548)

In [175]:
import re
from difflib import Differ, SequenceMatcher
from pprint import pprint

DEBUG = True

def atoi(num_str):
    '''
    Converts a string to an integer, or returns None.
    '''
    try:
        return int(num_str)
    except:
        pass
    return None

# phrases = ["BC546A/B/C...BC550A/B/C", "BC547A, BC5XB, C", "BC546A~BC546Z", "BC546/550/543", "BC547A/BC548B", "BC182,A,B", "BC546/D"]
phrases = ["BC547A, ABC, D"]
# This range pattern will find text that "looks like" a range.
range_pattern = re.compile(ur'(?P<start>[\w/]+)(?:\s*(\.{3,}|\~|\-+|to)\s*)(?P<end>[\w/]+)')
suffix_pattern = re.compile(ur'(?P<spacer>(?:,|\/)\s*)(?P<suffix>\w+)')
base_pattern = re.compile(ur'(?P<base>\w+)(?P<spacer>(?:,|\/)\s*)?(?P<suffix>\w+)?')
for phrase in phrases:
    inferred_phrases = set()
    final_set = set()
    m = re.search(range_pattern, phrase)
    if m:
        start = m.group("start")
        end = m.group("end")
        
        if DEBUG:
            print "[debug] Start: %s \t End: %s" % (start, end)
        
        # Use difflib to find difference. We are interested in 'replace' only
        seqm = SequenceMatcher(None, start, end).get_opcodes();
        for opcode, a0, a1, b0, b1 in seqm:
            if opcode == 'equal':
                continue
            elif opcode == 'insert':
                break
            elif opcode == 'delete':
                break
            elif opcode == 'replace':
                # NOTE: Potential bug if there is more than 1 replace
                start_diff = start[a0:a1]
                end_diff = end[b0:b1]
            else:
                raise RuntimeError, "[ERROR] unexpected opcode"


        if DEBUG: print "[debug] start_diff: %s \t end_diff: %s" % (start_diff, end_diff)

        # Check Numbers
        if atoi(start_diff) and atoi(end_diff):
            if DEBUG: print "[debug] Enumerate %d to %d" % (start_num, end_num)
            # generate a list of the numbers plugged in
            number_range = range(atoi(start_diff), atoi(end_diff) + 1)
            for number in number_range:
                new_phrase = start.replace(start_diff,str(number))
                # Produce the strings with the enumerated ranges
                inferred_phrases.add(new_phrase)

        # Second, check for single-letter enumeration
        if len(start_diff) == 1 and len(end_diff) == 1:
            if start_diff.isalpha() and end_diff.isalpha():
                def char_range(a, b):
                    '''
                    Generates the characters from a to b inclusive.
                    '''
                    for c in xrange(ord(a), ord(b)+1):
                        yield chr(c)
                
                if DEBUG: print "[debug] Enumerate %s to %s" % (start_diff, end_diff)
                letter_range = char_range(start_diff, end_diff)
                for letter in letter_range:
                    new_phrase = start.replace(start_diff,letter)
                    # Produce the strings with the enumerated ranges
                    inferred_phrases.add(new_phrase)
    else: inferred_phrases.add(phrase)
    if DEBUG: print "[debug] Inferred Phrases: \n  " + str(sorted(inferred_phrases))
    
    # Handle lists for each of the inferred phrases
    # NOTE: this only does the simple case of replacing same-length suffixes.
    # we do not handle cases like "BC546A/B/XYZ/QR"
    for phrase in inferred_phrases:
        first_match = re.search(base_pattern,phrase)
        if first_match: 
            base = re.search(base_pattern,phrase).group("base");
            final_set.add(base) # add the base (multiple times, but set handles that)
            if (first_match.group("suffix")):
                all_suffix_lengths = set()
                # This is a bit inefficient but this first pass just is here
                # to make sure that the suffixes are the same length
                for m in re.finditer(suffix_pattern, phrase):
                    suffix = m.group("suffix")
                    suffix_len = len(suffix)
                if len(all_suffix_lengths) == 1:
                    for m in re.finditer(suffix_pattern, phrase):
                        spacer = m.group("spacer")
                        suffix = m.group("suffix")
                        suffix_len = len(suffix)
                        if prev_suffix_len != suffix_len:
                            break # suffixes aren't same length

                        trimmed = base[:-suffix_len]
                        final_set.add(trimmed+suffix)
    if DEBUG: print "[debug] Final Set: \n  " + str(sorted(final_set))

    # NOTE: We make a few assumptions (e.g. suffixes must be same length), but
    # one important unstated assumption is that if there is a single suffix,
    # (e.g. BC546A/B), the single suffix will be swapped in no matter what.
    # In this example, it works. But if we had "ABCD/EFG" we would get "ABCD,AEFG"

[debug] Inferred Phrases: 
  ['BC547A, ABC, D']
[debug] Final Set: 
  ['BC547A']
